<th:block layout:fragment="body" th:remove="tag">
    <div class="container py-4 bg-primary text-black">
        <header class="pb-3 mb-4 border-bottom">
            <a href="#" class="d-flex align-items-center text-light text-decoration-none">
                <span class="fs-4" th:text="'Welcome '+ ${username} "></span>
            </a>
        </header>

        <!-- Error message if teacher is not found -->
        <div th:if="${error}" class="alert alert-danger" role="alert">
            <p th:text="'Teacher not found. Scan QR Code again. Reach out to your teacher for assistance.'"></p>
        </div>

        <!-- Show Hall Pass form only if teacher is found (no error) -->
        <div th:if="${teacher}" class="bg-light p-4 rounded">
            <h2 th:text="'Hall Pass for ' + ${teacher.firstname} + ' ' + ${teacher.lastname}"></h2>


            <div th:if="${activePass}">
                <h3>You have an active hall pass. Please check-in to end session</h3>
                <p><strong>Teacher:</strong> Mortensen</p>

                <form th:action="@{/mvc/bathroom/checkoutPass}" method="post">

                    <button type="submit" class="btn btn-primary">Check In</button>
                </form>

                <button type="button" class="btn btn-secondary mt-3" onclick="approveStudent()">I'm done</button>
            </div>
            <!-- Request form, hidden if there is an active pass -->
            <div th:if="${activePass == null}">
                <form th:action="@{/mvc/bathroom/requestPass}" method="post">
                    <input type="hidden" name="teacherId" th:value="${teacher.id}" />
                    <div class="form-group">
                        <label for="period">Period:</label>
                        <select id="period" name="period" class="form-control" required>
                            <option value="1">1</option>
                            <option value="2">2</option>
                            <option value="3">3</option>
                            <option value="4">4</option>
                            <option value="5">5</option>
                        </select>
                    </div>

                    <div class="form-group">
                        <label for="activity">Activity:</label>
                        <select id="activity" name="activity" class="form-control" required>
                            <option value="bathroom">Bathroom</option>
                            <option value="library">Library</option>
                            <option value="other">Other</option>
                        </select>
                    </div>

                    <button type="submit" class="btn btn-primary mt-3">Request Hall Pass</button>
                </form>
            </div>
        </div>
    </div>
</th:block>

<script>
    import { javaURI, fetchOptions } from '{{ site.baseurl }}/assets/js/api/config.js'

    // Get teacher name and student name from local storage or variables
    const teacherName = "Mortensen"; // Replace with dynamic teacher name if needed
    const studentName = localStorage.getItem("email"); // Assume stored from login
    const approveUrl = javaURI + '/approve';
    const adminUrl = javaURI + '/api/admin/add';

    const postOptions = {
        ...fetchOptions,
        method: 'POST',
    };

    // Function to approve student who is first in line
    function approveStudent() {
        const now = new Date();
        const hours = now.getHours();
        const minutes = now.getMinutes().toString().padStart(2, "0"); // Ensures 2 digits
        const seconds = now.getSeconds().toString().padStart(2, "0");
        const timeOut = `${hours}:${minutes}:${seconds}`;
        const timeIn = localStorage.getItem("timeIn");
        var timeSpentMinutes = 0;

        if (timeIn) {
            // Convert both timeIn and timeOut from "HH:MM:SS" to total seconds
            const [inHours, inMinutes, inSeconds] = timeIn.split(":").map(Number);
            const [outHours, outMinutes, outSeconds] = timeOut.split(":").map(Number);

            // Convert both times to total seconds since midnight
            const totalSecondsIn = inHours * 3600 + inMinutes * 60 + inSeconds;
            const totalSecondsOut = outHours * 3600 + outMinutes * 60 + outSeconds;

            // Calculate the difference in seconds
            const timeSpentSeconds = totalSecondsOut - totalSecondsIn;

            // Convert timeSpent from seconds to minutes (with 2 decimal places)
            const timeSpentMinutes = (timeSpentSeconds / 60).toFixed(2);

            console.log(`Time spent: ${timeSpentMinutes} minutes`);
        } else {
            console.log("No timeIn value stored in localStorage.");
        }

        const timeInOut = `${timeIn}-${timeOut}`;


        const times = {
            studentEmail: studentName,
            timeIn: timeInOut,
            averageDuration: timeSpentMinutes,
        };

        const queuer = {
            teacherName: teacherName,
            studentName: studentName,
        };

        fetch(adminUrl, {
            ...postOptions,
            body: JSON.stringify(times),
        })
            .then(response => {
                console.log(body);
                if (response.ok) {
                    console.log("Time added to database");
                }
                else {
                    alert("Failed to add time to database.");
                }
            })
            .catch(error => console.error("Error adding time to database:", error));


        fetch(approveUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => {
                if (response.ok) {
                    window.location.href = "{{site.baseurl}}/hallpass";
                }
                else {
                    alert("Failed to approve student.");
                }
            })
            .catch(error => console.error("Error approving student:", error));
        window.location.href = "{{site.baseurl}}/profile";
    }
</script>